# Crude Estimates of Aerodynamic Forces

## <font color="blue">Model</font>
Quasi-Steady model - Decompose the aerodynamic force linearly into the following components:<br>
(1) Translational lift force<br>
(2) Translational drag force<br>
(3) Rotational force<br>
(4) Added mass force<br>

<br>
_(References: Dickinson+[2008](http://arc.aiaa.org/doi/10.2514/1.29862); Chin and Lentink [2016](http://jeb.biologists.org/content/219/7/920))_

## <font color="blue">Assumptions</font>
<br>
_(Note: All the numbers are based on the kinematics (see the kinematic [notebook](https://github.com/Mipanox/ME303/blob/master/kinematics/kinematics.ipynb))_
<br>
* Neglect forces exerted by body, tail, and feet (at the moment)
* Wing is rectangular with aspect ratio of 2
<br><p style="text-indent: 30px">The half-wingspan (i.e. for a single wing) is taken to be $40$cm for both species. Hence the chord length is (uniformly) $20$cm.<br><br>
* Wing motion is sinusoidal with fixed frequency and amplitude
<br><p style="text-indent: 30px">1. Avocet: `amplitude` $\approx15$cm; `frequency` $\approx5$Hz $\approx30$rad/s
<br><p style="text-indent: 30px">2. Shoveler: `amplitude` $\approx10$cm; `frequency` $\approx4$Hz $\approx24$rad/s

### <font color="green">Scenarios</font>
Let's focus first on two scenarios during landing:
1. Steady flight with body aligned parallel to the horizontal
2. Stopping phase where the body (and so do the wings) is $\theta$ degrees tilted w.r.t. the horizontal
<br><p style="text-indent: 30px">$\theta\approx60^\circ$ and $\approx20^\circ$ for Avocet and Shoveler respectively.
<br><p style="text-indent: 30px">(Note: Assuming horizontal wind (or rest) velocity, this is also the angle of attack. Namely $\alpha\sim\theta$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
## parameters
### global
time_elapse = 0.


### Scenario 1

### Scenario 2


## <font color="Red">Comparison with body motion</font>
In the simplest approximation, we can quantify the dynamics of the bird by examining its body motion, in particular the acceleration. Upon comparing this with the estimated total forces above, **extra** forces that are required to match with the measurements can be deduced.